# Environment Setup

In [8]:
import pandas as pd  
import numpy as np  
import os
import json
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

In [11]:
load_dotenv() # Loading the env to fetch environment variables

KEY = os.getenv("OPENAI_API_KEY")

In [12]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.3)

# Importing the required libraries and modules from Langchain

In [14]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain 
from langchain.chains import SequentialChain 
from langchain.callbacks import get_openai_callback 
import PyPDF2

# Quiz Generation - Prompt and Chain Initialization

In [19]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [21]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response likeRESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}
"""

# Here, tone implies difficulty level



quizGen_prompt_template = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)


quiz_chain = LLMChain(llm=llm,prompt=quizGen_prompt_template,output_key="quiz",verbose=True)

# Answer Evalusation Template Initiliazation

In [22]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

quizEval_prompt_template = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

review_chain = LLMChain(llm=llm,prompt=quizEval_prompt_template,output_key="review",verbose=True)

# Sequential Chain Generation
Let us now generate a chain of prompts and answers for the quiz generation task. Here, both the above chains will be combined to generate a single chain and give us the final output as desired.

In [23]:
evaluation_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                        output_variables=["quiz","review"],verbose=True)

# Data Loading

In [28]:
file_path = '../data.txt'
with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]
Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.
The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised l

In [29]:
# Now, let us serialize the above python dictionary into a json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

# Callback Function for Quiz Generation
Now, we will define a callback function that will be used to generate the quiz questions and answers. This function will be called by the chain to generate the quiz questions and answers. By this, we can track the tokens used in the prompt and the answer and get the cost of the prompt and the answer. Read more here [here](https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking).

In [30]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "Simple"

In [31]:
with get_openai_callback() as cb:
    response=evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]
Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.
The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mini

Let us get the token usage of the prompt and the answer.

In [36]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost in USD:{cb.total_cost}")

Total Tokens:1343
Prompt Tokens:930
Completion Tokens:413
Total Cost in USD:0.002221


Let us now try to get the response from the chain and see our quiz.

In [41]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\nMachine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through un

In [40]:
quiz = response.get('quiz')
quiz = json.loads(quiz)
quiz

{'1': {'mcq': 'What is the main focus of machine learning?',
  'options': {'a': 'Data visualization',
   'b': 'Developing statistical algorithms',
   'c': 'Creating artificial neural networks',
   'd': 'Building robots'},
  'correct': 'b'},
 '2': {'mcq': 'Which field has machine learning NOT been applied to?',
  'options': {'a': 'Natural language processing',
   'b': 'Agriculture',
   'c': 'Music production',
   'd': 'Medicine'},
  'correct': 'c'},
 '3': {'mcq': 'What provides the mathematical foundations for machine learning?',
  'options': {'a': 'Data mining',
   'b': 'Computational statistics',
   'c': 'Mathematical optimization',
   'd': 'Artificial intelligence'},
  'correct': 'c'},
 '4': {'mcq': 'Which field focuses on exploratory data analysis through unsupervised learning?',
  'options': {'a': 'Machine learning',
   'b': 'Data mining',
   'c': 'Artificial intelligence',
   'd': 'Predictive analytics'},
  'correct': 'b'},
 '5': {'mcq': 'What framework provides a theoretical view

Let us now convert the response to a DataFrame and display the quiz.

In [42]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [44]:
quiz = pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What is the main focus of machine learning?,a: Data visualization | b: Developing statisti...,b
1,Which field has machine learning NOT been appl...,a: Natural language processing | b: Agricultur...,c
2,What provides the mathematical foundations for...,a: Data mining | b: Computational statistics |...,c
3,Which field focuses on exploratory data analys...,a: Machine learning | b: Data mining | c: Arti...,b
4,What framework provides a theoretical viewpoin...,a: Artificial intelligence | b: Data mining | ...,c


Now finally, we can export the above DataFrame to a csv file.

In [46]:
quiz.to_csv("ML_MCQ_Quiz.csv",index= False)

With this, we have successfully generated a quiz using Langchain and understood the process of generating a quiz using Langchain. We can now use this above code to take input from a user and generate a quiz based on the input using APIs and deploying the same web application on a cloud platform.